In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# read data and skip 'date' colume
df = pd.read_csv('./dataset/netflix-prize-data/combined_data_1.txt', 
                 header = None, names = ['user', 'rating'], 
                 usecols = [0,1])


In [11]:
def slice_movie(df):
    # find empty rows for slice movies 
    # dataset using movie id as user name for splite rating of each movies
    movies_interval = df[df['rating'].isna()]['user'].reset_index()
    movies_interval['index'] = movies_interval['index'].astype(int)
    
    # create list accroding to movie interval by np
    movie_id = 1
    movie_np = []
    last = None
    for i,r in movies_interval.iterrows():
        if last != None: 
            temp = np.full((1,r['index']-last-1), movie_id)
            movie_np = np.append(movie_np, temp)
            movie_id +=1
        last = r['index']
    # add last movie
    temp = np.full((1,len(df)-last-1), movie_id)
    movie_np = np.append(movie_np, temp)
    
    # removie movie slice row, and combine movie colume
    df = df[pd.notnull(df['rating'])]
    df['movie'] = movie_np.astype(int)
    
    return df

In [12]:
def get_users_df(df):
    # get user list and their rating count
    return df.groupby('user')['user'].agg(['count']).reset_index()

In [13]:
def filter_data_by_user(df,users_df, frac=0.1,count=50):
    # get test users
    test_users_df = users_df[users_df['count']>count].sample(frac=frac)
    # filter user by rating count
    cdf = df.merge(users_df, left_on='user',right_on='user')
    cdf = cdf[cdf['count']>count][['user','movie','rating']]
    
    train_set = cdf[~(cdf['user'].isin(test_users_df['user']))]
    test_set = cdf[cdf['user'].isin(test_users_df['user'])]
    return train_set, test_set


In [14]:
sliced_df = slice_movie(df)

/Users/wujingyi/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
sliced_df['rating']=sliced_df['rating'].astype(float)
sliced_df['movie']=sliced_df['movie'].astype(int)
sliced_df['user']=sliced_df['user'].astype(int)
print(sliced_df)

/Users/wujingyi/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/wujingyi/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


             user  rating  movie
1         1488844     3.0      1
2          822109     5.0      1
3          885013     4.0      1
4           30878     4.0      1
5          823519     3.0      1
6          893988     3.0      1
7          124105     4.0      1
8         1248029     3.0      1
9         1842128     4.0      1
10        2238063     3.0      1
11        1503895     4.0      1
12        2207774     5.0      1
13        2590061     3.0      1
14           2442     3.0      1
15         543865     4.0      1
16        1209119     4.0      1
17         804919     4.0      1
18        1086807     3.0      1
19        1711859     4.0      1
20         372233     5.0      1
21        1080361     3.0      1
22        1245640     3.0      1
23         558634     4.0      1
24        2165002     4.0      1
25        1181550     3.0      1
26        1227322     4.0      1
27         427928     4.0      1
28         814701     5.0      1
29         808731     4.0      1
30        

/Users/wujingyi/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
users_df = get_users_df(sliced_df)
train_set, test_set = filter_data_by_user(sliced_df,users_df,frac=0.1,count=50)
print(train_set)

             user  movie  rating
0         1488844      1     3.0
1         1488844      8     4.0
2         1488844     17     2.0
3         1488844     30     3.0
4         1488844     44     3.0
5         1488844     58     5.0
6         1488844     76     3.0
7         1488844     80     3.0
8         1488844     81     3.0
9         1488844     83     3.0
10        1488844    108     3.0
11        1488844    111     2.0
12        1488844    118     3.0
13        1488844    143     5.0
14        1488844    173     3.0
15        1488844    175     3.0
16        1488844    187     2.0
17        1488844    189     2.0
18        1488844    191     5.0
19        1488844    195     4.0
20        1488844    197     3.0
21        1488844    199     3.0
22        1488844    216     3.0
23        1488844    232     3.0
24        1488844    241     3.0
25        1488844    268     4.0
26        1488844    270     4.0
27        1488844    273     2.0
28        1488844    275     3.0
29        

In [18]:
train_set.to_csv(r'./dataset/cleaned_train_2.csv',index=False,sep=',')
test_set.to_csv(r'./dataset/cleaned_test_2.csv',index=False,sep=',')

In [ ]:
def make_sprase_matrix(df,users_df):
    np_matrix = np.zeros((df.shape[0], df.shape[1]))
    users = list(users_df['user']) #用户列表

    for user in users:
        user_list = df[df['user']== user] #获取全部用户评分
        np_matrix[i][user_list['movie'] -1] = user_list['rating'] #按index插入评分

    return sprase_matrix

In [ ]:
sprase_train = make_sprase_matrix(train_set,users_df)
np.savetxt('dataset/sprase_train_2.txt',sprase_train)
sprase_test = make_sprase_matrix(test_set,users_df)
np.savetxt('dataset/sprase_test_2.txt',sprase_test)